<a href="https://colab.research.google.com/github/dhanyavicky/Algorithm-Complexity/blob/main/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 02 Assignment

In class we discussed how *optimal substructure* enables *dynamic programming* approaches. To oversimplify (and some how inaccurately), dynamic programming is a greedy or a divide and conquer algorithm with some [memoization](https://en.wikipedia.org/wiki/Memoization) built-in.

A problem exhibits optimal substructure when its optimal solution can be built from optimal solutions to its subproblems. If a problem can be broken into smaller parts, and the best solution to the whole problem always includes the best solutions to those parts, then the problem exhibits optimal substructure. Dynamic programming works by solving subproblems first, storing their results, and then combining them to form the overall solution.

One good example of dynamic programming is finding the Maximum Weight Independent Set (MWIS) of a path graph. A graph $G=(V,E)$ with $|V|=n$ and $|E|=n-1$ that has a single component, is a path graph. In such graph, if
$$
V = \{v_1, v_2, \ldots v_{n-2}, v_{n-1}, v_n\}
$$
then
$$
E = \{ (v_1, v_2), (v_2,v_3), \ldots (v_{n-3}, v_{n-2}), (v_{n-2}, v_{n-1}), (v_{n-1}, v_n)\}
$$
Non-trivial path graphs have two or more vertices, ie, $n\ge 2$.

An indepedent set $S$ of a graph $G$ is a set of mutually exclusive vertices from $V$, i.e.,
$$
S = \{u, v: u\in V\ \text{and}\ v\in V\ \text{and}\ (u,v)\not\in E\}
$$
If every vertex in the graph is assigned a weight (not to be confused with edge weights in some graphs), the objective of the problem is to find heaviest subset of independent vertices. In general,
$$
S =
\begin{cases}
  V & \text{if } |V| = 1,\\
  \text{a proper subset of } V & \text{if } |V| > 1.
\end{cases}

$$

A path graph with $n$ vertices has $2^n$ subsets and only a few of them would qualify as independent sets. For small graphs, with $n<30$ maybe, we can attempt a brute-force solution. Compute all 10 billion subsets, discard those that are not independent, measure the total weight of the remaining subsets, and determine which one is the largest. A fast computer can process about 10,000 graphs per second, so this will take about 12 days to compute. Not lightning fast, but definitely manageable. A faster computer may even solve this problem in a couple of hours.

What if the path graph had 100 vertices -- that's not too big of a graph. In this case we have $2^{100}$ subsets or about 1,270,000,000,000,000,000,000,000,000,000 subsets (that's about $1.27\times 10^{30})$. A very fast computer processing one billion graphs per second will require abour 40 trillion years to compute. For perspective, the current scientific consensus about the age of the universe, based on measurements of the cosmic microwave background and expansion rate, is about 14 billion years. The solution to the MWIS of a path graph with 100 vertices will take 3,000 times longer than the age of the universe!

Rather than waiting to solve the problem in brute force while watching our Sun turn into a dark piece of frozen stellar ash, we tried a different approach: **what if the solution was handed to us?** In other words, the MWIS $S$ miraculously appears before us?

In this case we can make two simple and mutually exclusive observations regarding a *very special* vertex in $S$:

\begin{align*}
\text{either}\ &v_n\in S\\
\text{or}\ &v_n\not\in S
\end{align*}

If $v_n\not\in S$ then we can show that $S$ is a solution to the immediately smaller problem $G_{n-1}$. That's the path graph with the first $n-1$ vertices.

If $v_n\in S$ then $S$ is a solution to $G_{n-2} \cup \{v_n\}$. Here, $G_{n-2}$ is the path graph with the first $n-2$ vertices.

The same assumption we made for $G_n$ (the full path graph) can be made for the smaller problems. For example, if $v_n\not\in S$ and therefore $S$ is the MWIS solution for $G_{n-1}$, then we can look at the smaller problem $G_{\color{brown}m}$ with ${\color{brown}m}=n-1$, and repeat the process: the last vertex of $G_{\color{brown}m}$ (which is $v_{n-1}$) may or may not be in $S$, and so on.

This process leads to a recurrence relation about the maximum weight $W_n$ of the MWIS for a path graph with $n$ vertices, each with weight $w_i > 0$:

$$
W_n = \max\left(
    W_{n-1}, W_{n-2}+w_n
\right)
$$

In class, we also discussed how the *single-source shortest path* algorithm is an example of dynamic programming. The shortest path from a vertex $s$ to any vertex $v$ in a graph $G=(V,E)$ is given by
$$
\text{dist}(s,v) = \min_{{u,v}\in E} \left(
    \text{dist}(s,u) + w(u,v)
\right)
$$
where $w(u,v)$ is the weight of the edge between vertices $u$ and $v$.

The recurrence above tells that the shortest distance $s\rightsquigarrow v$ depends on the shortest distance  $s\rightsquigarrow u$ of its predecessor.

# Your assignment

Demonstrate that a plain greedy algorithm cannot give us the correct SSSP. You may consider a graph with 7 vertices as an example.

```python
_ = float('inf')

# Order of vertices: s, a, b, c, d, e, t
adj_matrix = [
    [0,   1,   2,   _,   _,   _,   _],   # s (source)
    [_,   0,   _,   _,   _,   _, 100],   # a
    [_,   _,   0,   2,   _,   _,   _],   # b
    [_,   _,   _,   0,   2,   _,   _],   # c
    [_,   _,   _,   _,   0,   2,   _],   # d
    [_,   _,   _,   _,   _,   0,   2],   # e
    [_,   _,   _,   _,   _,   _,   0]    # t (target)
]
```

This is a qualitative assignment. No need for code or math proofs. Just persuavive arguing using the provided example.

**Introduction** Dhanya Sridhar

In this assignment, I aim to demonstrate why a plain greedy algorithm cannot be relied upon to solve the Single Source Shortest Path (SSSP) problem. Although greedy methods feel intuitive and efficient, they often fail because they optimize only locally at each step without considering the global structure of paths. Using the provided 7-vertex example, I will show how a greedy rule leads to a dramatically suboptimal solution.

**Example Graph Setup**

https://drive.google.com/file/d/1m8nPOC4m87sD9mSzmXLmc0DfCVutsZCl/view?usp=share_link

We are given seven vertices: s, a, b, c, d, e, t.
The edges are defined as:
	•	s \to a = 1, s \to b = 2
	•	a \to t = 100
	•	b \to c = 2, \; c \to d = 2, \; d \to e = 2, \; e \to t = 2

   ** Greedy Rule and Its Outcome**

The greedy rule says: “At each step, from the current vertex, choose the outgoing edge with the smallest weight.”
	•	At s, it compares s \to a (1) vs. s \to b (2). It picks s \to a because 1 is smaller.
	•	From a, the only outgoing edge is a \to t (100).

Thus, the greedy path is:
s \to a \to t \quad \text{with total cost } 1 + 100 = 101

**The True Shortest Path**

Instead, consider starting with s \to b. This leads into the chain:
s \to b \to c \to d \to e \to t
with cost: 2 + 2 + 2 + 2 + 2 = 10.

This is the actual shortest path.

**Why the Greedy Approach Fails**
	•	Short-sightedness: The greedy choice at s appears best locally (1 vs. 2), but it traps the path into a very expensive edge later (100).
	•	Lost Opportunity: The slightly more expensive first step (2) enables a much cheaper chain overall.
	•	No Global Awareness: Greedy ignores how today’s choice affects tomorrow’s costs.

 ** Broader Perspective**
	•	This failure highlights that local optimality does not guarantee global optimality.
	•	Algorithms like Dijkstra’s are also sometimes called “greedy,” but crucially, they maintain global distance information and select the next vertex with the smallest total distance so far—not just the smallest outgoing edge. This is why Dijkstra’s succeeds where the plain greedy approach fails.

**  Conclusion**

Through this 7-vertex example, we see that the plain greedy strategy produces a path costing 101 instead of the optimal 10. This illustrates a fundamental limitation: SSSP requires algorithms that balance local steps with global awareness. Greedy alone is not enough. Correct approaches (like Dijkstra’s or Bellman–Ford) succeed precisely because they account for the global structure of shortest paths.

# Reading
[Chapter 3](https://jeffe.cs.illinois.edu/teaching/algorithms/book/03-dynprog.pdf) from Jeff's book

